In [1]:
from datatalker.ogdp import OGDProxy

ogd = OGDProxy(api_key="579b464db66ec23bdd00000112e7f83cb5554f526700caab1cf0139d")

/data102/vijay/datatalker/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
catalogs = ogd.get_all(ogd.catalogs)

In [9]:
# sudo docker volume create --driver local --opt device=/data102/.docker/volumes --opt type=none  mongodb_data
# docker run --name mongodb -d -p 27017:27017 -v mongodb_data:/data/db mongodb/mongodb-community-server

In [2]:
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")

In [3]:
dt_db = client["datatalker"]
ogd_catalogs = dt_db["ogd_catalogs"]

In [36]:
ogd_catalogs.insert_many([
    c
    for data in catalogs
    for c in data["data"]["rows"]
])

InsertManyResult([ObjectId('6813021160284fbf048a0f65'), ObjectId('6813021160284fbf048a0f66'), ObjectId('6813021160284fbf048a0f67'), ObjectId('6813021160284fbf048a0f68'), ObjectId('6813021160284fbf048a0f69'), ObjectId('6813021160284fbf048a0f6a'), ObjectId('6813021160284fbf048a0f6b'), ObjectId('6813021160284fbf048a0f6c'), ObjectId('6813021160284fbf048a0f6d'), ObjectId('6813021160284fbf048a0f6e'), ObjectId('6813021160284fbf048a0f6f'), ObjectId('6813021160284fbf048a0f70'), ObjectId('6813021160284fbf048a0f71'), ObjectId('6813021160284fbf048a0f72'), ObjectId('6813021160284fbf048a0f73'), ObjectId('6813021160284fbf048a0f74'), ObjectId('6813021160284fbf048a0f75'), ObjectId('6813021160284fbf048a0f76'), ObjectId('6813021160284fbf048a0f77'), ObjectId('6813021160284fbf048a0f78'), ObjectId('6813021160284fbf048a0f79'), ObjectId('6813021160284fbf048a0f7a'), ObjectId('6813021160284fbf048a0f7b'), ObjectId('6813021160284fbf048a0f7c'), ObjectId('6813021160284fbf048a0f7d'), ObjectId('6813021160284fbf048a0f

In [38]:
unique_keys = set()
for doc in ogd_catalogs.find():
    unique_keys.update(doc.keys())

In [40]:
unique_keys

{'_freq_wise_not_updated',
 '_freq_wise_schedule',
 '_freq_wise_total_resources',
 '_freq_wise_updated',
 '_id',
 'api_request_count',
 'body:value',
 'changed',
 'created',
 'data_source',
 'data_time_period_from',
 'data_time_period_to',
 'field_asset_jurisdiction:name',
 'field_ds_govt_type',
 'field_group_name:name',
 'field_ministry_department:name',
 'field_search_keywords',
 'field_sector:name',
 'field_state_department:name',
 'frequency',
 'frequency_data_series',
 'from_api',
 'govt_type',
 'having_api',
 'high_value_dataset',
 'is_api_available',
 'is_priced',
 'is_webservice',
 'keywords',
 'nid',
 'node_alias',
 'ogdp_custom_field',
 'ogdp_view_count',
 'ogpl_module_domain_access',
 'ogpl_module_domain_name',
 'published_date',
 'search_api_datasource',
 'search_api_id',
 'search_api_language',
 'title',
 'uuid',
 'vid',
 'view_count'}

In [4]:
ogd_catalogs.count_documents({})

12451

In [4]:
catalog_uuids = ogd_catalogs.distinct("uuid")
len(catalog_uuids)

12451

In [ ]:
from multiprocessing import cpu_count, Pool
import requests

def get_data(uuid: str):
  return requests.get(f"https://www.data.gov.in/backend/dmspublic/v1/catalog/{uuid}?format=json").json()

with Pool(cpu_count()) as pool:
  args = [
    (uuid)
    for uuid in catalog_uuids[:5]
  ]
  responses = pool.starmap(get_data, args)


TypeError: get_data() takes 1 positional argument but 36 were given